<a href="https://colab.research.google.com/github/mathildabineli/chatbot/blob/main/First_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -qU "langchain[mistralai]"

In [7]:
MISTRAL_API_KEY = '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'

In [8]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
            model_name="mistralai/mistral-small-3.1-24b-instruct:free",
            base_url="https://openrouter.ai/api/v1",
            api_key= MISTRAL_API_KEY,
            temperature=0.0,
        )


In [9]:
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 643, 'completion_tokens': 10, 'total_tokens': 653}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'model': 'mistralai/mistral-small-3.1-24b-instruct:free', 'finish_reason': 'stop'}, id='run--05418c18-c610-4af6-b3f8-613cc4bc8c5b-0', usage_metadata={'input_tokens': 643, 'output_tokens': 10, 'total_tokens': 653})

In [10]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! Nice to meet you. How can I assist you today? Let's chat about anything you'd like. 😊", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 9, 'completion_tokens': 28, 'total_tokens': 37, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'model': 'mistralai/mistral-small-3.1-24b-instruct:free', 'finish_reason': 'stop'}, id='run--a05d7e9a-6e01-472e-b1a1-538b95e44288-0', usage_metadata={'input_tokens': 9, 'output_tokens': 28, 'total_tokens': 37})

In [11]:
llm.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have the ability to know or remember personal information about individuals. If you'd like, you can share your name with me, and I'll be happy to use it in our conversation!", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 9, 'completion_tokens': 46, 'total_tokens': 55, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'model': 'mistralai/mistral-small-3.1-24b-instruct:free', 'finish_reason': 'stop'}, id='run--dc16f072-8ff0-412c-ab33-11c71b4637f4-0', usage_metadata={'input_tokens': 9, 'output_tokens': 46, 'total_tokens': 55})

In [12]:
from langchain_core.messages import AIMessage

llm.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I was just told that your name is Bob. Is there something specific you would like to do or talk about?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 30, 'completion_tokens': 24, 'total_tokens': 54, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'model': 'mistralai/mistral-small-3.1-24b-instruct:free', 'finish_reason': 'stop'}, id='run--82dfcd98-d733-4c30-8800-b4d27ab7e794-0', usage_metadata={'input_tokens': 30, 'output_tokens': 24, 'total_tokens': 54})

In [14]:
llm.invoke(
    [
        HumanMessage(content="yeah!can you talk about the digestive system in 3 sentences?"),
        AIMessage(content="Hello Bob! How can I assist you today?"),


    ]
)

AIMessage(content=' The digestive system is a complex network of organs that work together to break down food, extract nutrients, and eliminate waste. It begins at the mouth and ends at the anus, with key organs including the stomach, liver, pancreas, gallbladder, small intestine, and large intestine. The process involves both mechanical and chemical digestion, facilitated by enzymes and acids.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 664, 'completion_tokens': 74, 'total_tokens': 738}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'model': 'mistralai/mistral-small-3.1-24b-instruct:free', 'finish_reason': 'stop'}, id='run--81beac2f-fee7-45fd-b856-63b136754be1-0', usage_metadata={'input_tokens': 664, 'output_tokens': 74, 'total_tokens': 738})

In [15]:
pip install langgraph

In [18]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [19]:
config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Bob! Nice to meet you. How can I assist you today?


In [20]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I was just told that your name is Bob. Is there something specific you would like to talk about or do?
